# Modelo de Machine Learning para predecir el porcentaje de Similitud en datos. 

## SoftConsulting SA.

### Autor: Isaac Reyes

#### Librerias embeddings, transformers, etc pre instaladas:

In [1]:
import random
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

##### Usar GPU

In [2]:
#GPU si está disponible
device = 'cuda' if SentenceTransformer().device.type == 'cuda' else 'cpu'
print(f"Usando device: {device}")

Usando device: cpu


###### CODIGO

In [ ]:
#2.Definir tus grupos de sinónimos (DataFrame 1 sintético)
ramo_dic = {
    "VEHICULOS": [
        "inclusion vehiculos",
        "inclusion autos",
        "inclusion coches",
        "inclusion carros"
    ],
    "EQUIPOS ELECTRONICOS": [
        "inclusion equipos electronicos",
        "inclusion EE",
        "inclusion E Equipos",
        "inclusion equip e",
        "inclusion equipos electr"
    ],
    #aqui para mas ejemplos con zURICH más ramas aquí...
}
#df1:cada fila es un ramo con su lista concatenada
rows = []
for ramo, opts in ramo_dic.items():
    rows.append({
        "ramo": ramo,
        "concatenado": "; ".join(opts)
    })
df1 = pd.DataFrame(rows)
print("DataFrame 1 (sinónimos):")
print(df1, "\n")

DataFrame 1 (sinónimos):
                   ramo                                        concatenado
0             VEHICULOS  inclusion vehiculos; inclusion autos; inclusio...
1  EQUIPOS ELECTRONICOS  inclusion equipos electronicos; inclusion EE; ... 



In [4]:
#3. Generar DataFrame 2 (Asuntos sintéticos) con la etiqueta de ramo
def make_subject(ramo, opts):
    #aleatoriamente un sinónimo y lo inserta en una frase
    ejemplo = random.choice(opts)
    templates = [
        f"Por favor ayudar con el trámite {ejemplo}",
        f"Solicitud de {ejemplo}",
        f"Necesito asistencia para {ejemplo}",
        f"¿Pueden gestionar {ejemplo}?"
    ]
    return random.choice(templates)

n_per_group = 500  #total +1000 ejemplos
rows2 = []
for ramo, opts in ramo_dic.items():
    for _ in range(n_per_group):
        subj = make_subject(ramo, opts)
        rows2.append({
            "Asunto": subj,
            "ramo_true": ramo
        })
# añadir negativos (otros asuntos que no coinciden) si quieres
df2 = pd.DataFrame(rows2).sample(frac=1).reset_index(drop=True)
print("DataFrame 2 (Asuntos):")
print(df2.head(), "\n")


DataFrame 2 (Asuntos):
                                              Asunto             ramo_true
0  Por favor ayudar con el trámite inclusion equi...  EQUIPOS ELECTRONICOS
1                      Solicitud de inclusion carros             VEHICULOS
2          Necesito asistencia para inclusion coches             VEHICULOS
3             ¿Pueden gestionar inclusion vehiculos?             VEHICULOS
4         Necesito asistencia para inclusion equip e  EQUIPOS ELECTRONICOS 



In [5]:
#4.Cargar modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

In [7]:
#5. Obtener embeddings
emb_df1 = model.encode(df1['concatenado'].tolist(), convert_to_numpy=True, show_progress_bar=True)
emb_df2 = model.encode(df2['Asunto'].tolist(), convert_to_numpy=True, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [8]:
#6.Para cada Asunto, calcular similitud contra cada concatenado y predecir ramo
sims = cosine_similarity(emb_df2, emb_df1)  #shape (len(df2), len(df1))
pred_indices = np.argmax(sims, axis=1)
df2['ramo_pred'] = df1.loc[pred_indices, 'ramo'].values
df2['sim_max'] = sims[np.arange(len(df2)), pred_indices]

In [9]:
#7.Evaluar precisión en sintético
acc = accuracy_score(df2['ramo_true'], df2['ramo_pred'])
print(f"Accuracy sintético: {acc*100:.2f}%")

Accuracy sintético: 88.80%


In [9]:
#8. Mostrar ejemplos
print("\nAlgunos ejemplos de predicción:")
print(df2[['Asunto', 'ramo_true', 'ramo_pred', 'sim_max']].head(10))


Algunos ejemplos de predicción:
                                              Asunto             ramo_true  \
0  Por favor ayudar con el trámite inclusion equi...  EQUIPOS ELECTRONICOS   
1                      Solicitud de inclusion carros             VEHICULOS   
2          Necesito asistencia para inclusion coches             VEHICULOS   
3             ¿Pueden gestionar inclusion vehiculos?             VEHICULOS   
4         Necesito asistencia para inclusion equip e  EQUIPOS ELECTRONICOS   
5             ¿Pueden gestionar inclusion E Equipos?  EQUIPOS ELECTRONICOS   
6  Por favor ayudar con el trámite inclusion equi...  EQUIPOS ELECTRONICOS   
7                      Solicitud de inclusion carros             VEHICULOS   
8              Solicitud de inclusion equipos electr  EQUIPOS ELECTRONICOS   
9  Necesito asistencia para inclusion equipos ele...  EQUIPOS ELECTRONICOS   

              ramo_pred   sim_max  
0  EQUIPOS ELECTRONICOS  0.437551  
1             VEHICULOS  0.600696  


In [10]:
#9. Función para consultar nuevos asuntos
def predict_similaridad(asuntos:list):
    emb_new = model.encode(asuntos, convert_to_numpy=True)
    sim_matrix = cosine_similarity(emb_new, emb_df1)
    results = []
    for i, asunto in enumerate(asuntos):
        idx = np.argmax(sim_matrix[i])
        results.append({
            "Asunto": asunto,
            "ramo_pred": df1.loc[idx, 'ramo'],
            "sim_max (%)": sim_matrix[i, idx]*100
        })
    return pd.DataFrame(results)

#Ejemplo de uso
nuevos = [
    "Ayuda urgente para inclusion de autos",
    "Por favor gestionar inclusion EE",
    "Consulta sobre otro trámite"
]
print("\nPredicción sobre nuevos Asuntos:")
print(predict_similaridad(nuevos))


Predicción sobre nuevos Asuntos:
                                  Asunto  ramo_pred  sim_max (%)
0  Ayuda urgente para inclusion de autos  VEHICULOS    61.228347
1       Por favor gestionar inclusion EE  VEHICULOS    42.351627
2            Consulta sobre otro trámite  VEHICULOS    25.141025
